In [0]:
df = spark.read.format("csv").load("/mnt/devkivaloansdl/bronze/loan_themes_by_region.csv", header=True, inferSchema=True)
display(df)

In [0]:
df.createOrReplaceTempView("vw_loan_themes_by_region")

In [0]:
%sql
SELECT COUNT(1) FROM vw_loan_themes_by_region

In [0]:
%sql
SELECT * FROM vw_loan_themes_by_region LIMIT 100

In [0]:
df_select_required_columns = spark.sql("SELECT `Partner ID` as partner_id, `Field Partner Name` as field_partner_name, `Loan Theme Type` as loan_theme_type, `Loan Theme ID` as loan_theme_id, `country` as country, `region` as region, `amount` as amount, `LocationName` as LocationName,`rural_pct` as rural_pct from vw_loan_themes_by_region")

display(df_select_required_columns)

In [0]:
df_select_required_columns.write.format("delta").mode("overwrite").save("/mnt/devkivaloansdl/silver/loan_themes_by_region")

In [0]:
#delete the UNMANAGED delta table
dbutils.fs.rm("/mnt/devkivaloansdl/silver/loan_themes_by_region", recurse=True)

In [0]:
# Create a DELTA table on the silver schema
df_select_required_columns.write.format("delta").mode("overwrite").saveAsTable("silver.loan_themes_by_region")

In [0]:
# Create a parquet file on the silver layer
df_select_required_columns.write.format("parquet").mode("overwrite").save("/mnt/devkivaloansdl/silver/loan_themes_by_region_parquet")

In [0]:
# Validate the parquet file and table
display(spark.read.parquet("/mnt/devkivaloansdl/silver/loan_themes_by_region_parquet"))

In [0]:
display(spark.read.table("silver.loan_themes_by_region"))

In [0]:
%sql
SELECT COUNT(1) FROM silver.loan_themes_by_region